In [1]:
#Libraries used
import pandas as pd #for the data frame
import numpy as np #used for filtering and some arithmatic. 

In [2]:
#Import the data
df_import = pd.read_csv('data/pbp/raw/pbp_query_20162017.csv',low_memory=False) 

In [3]:
#The first thing to do is reduce the number of columns then rows that won't be used
df = df_import[['season', 'game_id', 'game_date', 'event_index', 'game_period',
                'game_seconds', 'event_type', 'event_description', 'event_detail',
                'event_team', 'coords_x', 'coords_y', 'home_team', 'away_team',
                'home_score', 'away_score', 'game_strength_state', 'event_distance',
                'event_angle']]
even_strength = df.game_strength_state.isin(['5v5','4v4','3v3'])
play_type     = df.event_type.isin(['FAC','SHOT','HIT','BLOCK','MISS','GIVE','TAKE','GOAL'])
no_coords     = ~df.coords_x.isna()
df            = df[even_strength & play_type & no_coords]

In [4]:
#Flipping the x/y coordinates for away team in 1st/3rd period and Home 2nd period.
#This makes sure that everything orients the same way. 
away_flip = (df.event_team == df.away_team) & (df.game_period.isin([1,3]))
home_flip = (df.event_team == df.home_team) & (df.game_period == 2)
home_cond = (df.logat == 1)
#Come back to this
df['x_flip'] = np.where((away_flip | home_flip) & home_cond ,-1*df.coords_x,df.coords_x)
df['y_flip'] = np.where((away_flip | home_flip) & home_cond ,-1*df.coords_y,df.coords_y)
df['x_flip'] = np.where((away_flip | home_flip) !& home_cond ,-1*df.coords_x,df.coords_x)
df['y_flip'] = np.where((away_flip | home_flip) !& home_cond ,-1*df.coords_y,df.coords_y)
#This moves the 0,0 point from center ice to the center of the goal on the red line
df['x_goaline'] = df.x_flip + 89
#How to get the distance
df['new_distance'] = ((df['x_goaline']**2) + (df['y_flip']**2))**(1/2)

In [116]:
df['logat'] = np.where(df.game_id.isin(get_lst),1,0)

In [43]:
check_lst = ['event_team','home_team','game_period','game_seconds',
             'coords_x','coords_y','x_flip','y_flip','x_goaline',
             'event_distance','new_distance','event_type','event_description']
df_rc = df[check_lst].copy()
df_rc['x_wtf_2'] = 89 - df.x_flip
df_rc['x_wtf'] = df.x_goaline**2
df_rc['y_wtf'] = df.y_flip
df_rc['new_distance_2'] = ((df_rc['x_wtf_2']**2) + (df_rc['y_flip']**2))**(1/2)

In [53]:
#df_rc.sample(10)

In [71]:
hometeam = df.event_team == df.home_team
shots    = df.event_type == 'SHOT'
first_period = df.game_period == 1
df_1 = df[hometeam & shots & first_period]

In [89]:
game_lst = []
team_lst = []
mean_lst = []
for game in df_1.game_id.unique():
    df_temp = df_1[df_1.game_id==game]
    game_lst.append(game)
    team_lst.append(df_temp.home_team.iloc[0])
    mean_lst.append(df_temp.coords_x.mean())
    
new_df = pd.DataFrame(list(zip(game_lst,team_lst,mean_lst)),
                      columns=['Game','Team','avg'])

In [113]:
testing = new_df[new_df.avg > 0].Game.tolist()

In [111]:
get_lst = new_df_1.Game.tolist()